<a href="https://colab.research.google.com/github/mifm/examples/blob/master/XRSite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install py_wake

In [2]:
import xarray as xr
from py_wake.site import XRSite
from py_wake.examples.data.hornsrev1 import Hornsrev1Site

# import a site example just to get some probability data
site = Hornsrev1Site()

# create your custom data set
ds = xr.Dataset(
    data_vars={'P': (['wd', 'ws'], site.local_wind(0,0).P),
               'TI': site.local_wind(0,0).TI},
    coords={'wd': site.local_wind(0,0).wd,
            'ws': site.local_wind(0,0).ws})

# create a site from a custom data set
mysite = XRSite(ds)